# Statistical Feature Extractor
**Author:** [Divyanshu Raghuwanshi](https://www.linkedin.com/in/divyanshu-raghuwanshi-85037b160/)<br>
**Date created:** 2020/16/05<br>

In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
from statistics import variance 
from scipy import stats
from scipy.fftpack import fft, ifft
import matplotlib as mpl
#from agnfunctions import mkdir_p
import h5py
import itertools # To create chain of lists of diff category features(optional) 
%matplotlib inline

In [2]:
#FILE_TYPES = ['Healthy','S_Imbalance','C_Imbalance','D_Imbalance', 'Misalign', 'Ped_Loose']   #['U_Damp','NU_Damp','L_Sensors','L_Casing','L_CasingU_Damp','L_CasingNU_Damp']
#RPM = ['750', '1500', '2250', '3000', '3250']  #input('Input RPM ')

FEATURE_NAME = ['CVX_Ampl','CVY_Ampl','SVX','SVY']   #['Speed',,'SVX_1X_Ampl','SVX_1X_Phase','SVX_2X_Ampl','SVX_2X_Phase','SVX_3X_Ampl','SVX_3X_Phase','SVY_1X_Ampl','SVY_1X_Phase','SVY_2X_Ampl','SVY_2X_Phase','SVY_3X_Ampl','SVY_3X_Phase']
Label_code = {'Healthy':0,'S_Imbalance':1,'C_Imbalance':2, 'D_Imbalance':3, 'Misalign':4, 'Ped_Loose':5}   #{'U_Damp':3,'NU_Damp':4,'L_Sensors':5,'L_Casing':6,'L_CasingU_Damp':7,'L_CasingNU_Damp':8}
NEW_FEATURE_NAME=['Mean', 'STD', 'Var', 'RMS', 'Kurtosis']   #['Avg.Speed', 'AbsMax', 'CoSkew', 'CrestF', 'MarginF', 'ShapeF', 'ImpulseF','A_Fact','B_Fact','Label']

In [3]:
import errno    
import os

def mkdir_p(path):
    try:
        os.makedirs(path)
    except OSError as exc:  # Python ≥ 2.5
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else:
            raise

In [4]:
def prepend(list, str): 
      
    # Using format() 
    str += '{0}'
    list = [str.format(i) for i in list] 
    return(list)

In [6]:
df_fullCatList = []  

outname = 'MeggitSummary.xlsx'
output_dir = "D:\\kit.me\\LSTM Codes and Data\\Summary_Data" #"C:\\Users\\hp\\AGN\\FeaturesExp"
outfname = output_dir +'/'+ outname 


mkdir_p(output_dir)

partCatWriter = pd.ExcelWriter(outfname, engine='xlsxwriter')
df_lst = []    
    
df_feature = pd.DataFrame()    
        
        
infname = 'D:\\kit.me\\working_data\\Data\\Summary_Data\\MeggitSummary.xlsx'
xlsfile = pd.ExcelFile(infname)
dframe = xlsfile.parse('Full Raw')

N = dframe[0].count()
W_Size = 200        # 10 minutes observation

#df_label=pd.DataFrame(columns=['Label'])

for i in range(18):
    if(i==1 or i==2 or i==3 or i==4):

        wf_dframe = dframe[i]
        labbel=dframe[17]

        nfn=[NEW_FEATURE_NAME]

        o=0
        
        df_feature1 = pd.DataFrame(columns=nfn) #, 'AbsMax', 'CoSkew', 'Kurtosis', 'CrestF', 'MarginF', 'ShapeF', 'ImpulseF','A_Fact','B_Fact','Label'])
        
        for idx in range(0, N, W_Size):
            if(o<6):
                if(labbel[idx]==labbel[idx+W_Size-1]==o):
                    part_dframe = dframe[i][idx:idx+W_Size]
                    N_part = part_dframe.count()
                    part_dlist = part_dframe.tolist()

                    part_mean = part_dframe.mean()
                    part_std = part_dframe.std()
                    part_var = variance(part_dframe)
                    part_RMS = np.sqrt(np.sum(np.square(part_dlist))/N_part)
                    part_kurtosis = np.sum(np.subtract(part_dlist,part_mean)**4)/((N_part-1)*part_std**4)


                    Label_c = o
                    
                    #df_label[0][x]=o
                    #x+=1
                    #df_label.append(pd.Series([o], index=['Label']), ignore_index=True)
                    
                    df_feature1 = df_feature1.append(pd.Series([part_mean, part_std, part_var, part_RMS, part_kurtosis], 
                                                             index=nfn), ignore_index=True)

                else:
                    o=o+1
                    continue
            
        df_feature=pd.concat([df_feature, df_feature1], axis=1, sort=False)
        
df_label=pd.DataFrame(columns=['Label'])
labbel=dframe[17]
o=0
for idx in range(0, N, W_Size):
    if(o<6):
        if(labbel[idx]==labbel[idx+W_Size-1]==o):
            df_label=df_label.append(pd.Series([o], index=['Label']), ignore_index=True)
            
        else:
            o=o+1
            continue
            
df_feature=pd.concat([df_feature, df_label], axis=1, sort=False)



#df_feature=pd.concat([df_feature, df_label], axis=1, sort=False)
#print(df_label)

df_lst.append(df_feature.values)   #Stores list of 'df_feature' dataframes. Each df_feature1 dataframe saves one feature(say CVX_Ampl..) and df_feature saves all features
                 
df_fullCatList.append(df_lst)  #Saves list of 'df_lst' lists. Each 'df_lst' saves complete file


for dataframe in df_lst:
    dataframe = pd.DataFrame(dataframe)
    dataframe.to_excel(partCatWriter, sheet_name='Statistical_Features_basic',  startrow=0 , startcol=0) 
partCatWriter.save()      
partCatWriter.close()



In [9]:
h5f = h5py.File('data.h5', 'w')
In [5]: h5f.create_dataset('dataset_1', data=fullCatArray_Cln)

<HDF5 dataset "dataset_1": shape (506, 240), type "<f8">